In [1]:
import pandas as pd
import numpy as np

# Part 1 Basic Map Calibration

### 1. Load the Open Street Map (OSM) of Fremont    
    a. Donwload OSM file   
    b. In Aimsun: File -> Import -> OpenStreetMap Files

### 2. Match the satellite map tile of Fremont with the OSM   
    a. Download the satellite map tile of the corresponding area from google map satellite view    
    b. In Aimsun: File -> Import -> Image File    
    c. (Less precise) Double click the Image and open Positioner tab, choose 4 Reference points in the Map and the corresponding satellite map tile.    
    d. (Recommended, more precise) Alternatively, rotate the satellite map tile manually before loading into Aimsun. Then in the Basic tab in Aimsun, set ‘Width’ and ‘Height’ of Physical Size manually.    

### 3. Revise the OSM based on the satellite map tile
    a. Edit the number of lanes   
    b. Edit the intersections   
    c. Add stop signs   
    d. Remove the non connected components, rails.   
    e. Most basic editing operations can be found in Aimsun Tutorial One.

### 4. Using scripts to load speed limit, capacity of sections
    a. Be careful about the unit of speed limit must be km/h
    b. One might change cost functions, but this requires a lot of data

```python
''' This is a Python 2 code, can be executed only in Aimsun '''
import csv
import os

def get_object(eid):
    return model.getCatalog().findObjectByExternalId(eid)

file = open("...")
reader = csv.reader(file)

for line in reader:
#     name = 'detector' + line[0] # line[0] should be object id 
#     print name
    
    target = get_object(str(line[0])) #  external id of the road section

    if target != None:
        print('Before, Section EID: {}, Name: {}, Speed: {}, Capacity: {}, Slope: {}'.format(target.getExternalId(), target.getName(), target.getSpeed(), target.getCapacity(), target.getSlope(0)))
        target.setSpeed(line[1])
        target.setCapacity(line[2])
        #target.setName(line[3])
#         print 'After, Section EID: %i, Name: %s, Speed: %i, Capacity: %i, Slope: %i' % (target.getExternalId(), target.getName(), target.getSpeed(), target.getCapacity(), target.getSlope(0))
    else:
        model.reportError("Set Section Parameters", "The script must be launched from a Section context menu")
```

input file format:

    line[0]: external id of the road section
    line[1]: speed limit
    line[2]: capacity
        
    Below are the information which are less important but still can be imported into aimsun but we can leave it as default:
    
    line[3]: rename the road section 
    
    an example are showing below

In [3]:
section_parameter_file = pd.DataFrame({'eid':[1000,1111,1234]
                       ,'speed':[25,45,65],'capacity':[150,200,300],'name':['roada','roadb','roadc']})

In [4]:
section_parameter_file

,eid,speed,capacity,name
0,1000,25,150,roada
1,1111,45,200,roadb
2,1234,65,300,roadc


### 5. Add Vehicle Type to differential Navigational Apps Users with Non-Navigational Apps Users   
    a. Add a new “Vehicle Type” in Vehicles.   
    b. Double click the new vehicle type and change necessary attributes.

***
# Part 2 Data Loading


<center>Data Summary</center>

| Data                      | Description                              | Note                    |
|:--------------------------|:-----------------------------------------|:------------------------|
| [1. Centroid](#centroid)     | One csv file containing: <br> **centroid_name**, **centroid_lat**, **centroid_long**, <br> **Generate_Section_eID**, **Absort_Section_eID**| 1. The **centroid_name** should be unique & consistent with the "OD Matrix data" <br> 2. The longitude and latitude should be **EPSG:32610** values |
| [2. OD Matrix](#od)          | Two csv containing the total vehicle count from one <br> centroid to another. | 1. The **centroid_name** should be unique & consistent with the "Centroid data" <br> 2. Two csv files `externel to externel` (commuter) and <br> `Non-externel to externel` (resident) are needed.
| [3. Detector](#detector)     | One csv file containing: **detector_name**, **section_eID** | 1. The **detector_name** should be unique & consistent with the "Real Data Set" |
| [4. Real Data Set](#rds)     | One csv file containing data collected by all the detectors in different time: <br> `detector_name`, `data_collected_time`, `count`, `speed`, etc    | 1. The **detector_name** should unique & be consistent with the "Detector" <br> 2. Vehicle count is the most important data, we might need `speed` data <br> in dynamic simulation later    |
| [5. Control Plan](#control)  | Several PDF files ||

### 0. Export the Aimsun network to Shapefile 
1. Run the following script in Aimsun to set `Externel ID` to `Aimsun Primary ID` (which is unique)

```python
''' This is a Python 2 code, can be executed only in Aimsun '''
for i in range(200000):
	Object = model.getCatalog().find(i)
	if Object != None:
		Object.setExternalId(str(Object.getId()))
```
2. In Aimsun: File -> Export -> GIS
3. Import the network into Shapefile, and generate data.

<a id='centroid'></a>
### 1. Add internal/external centroids to generate cars   <font color=red size=3>Data Format for Data Collection Team Reference</font>   
- It is one csv file for all internel/externel centroids.
- The file contains `Centroid_name`, `Centroid_Longitude`, `Centroid_Latitude`, `in_or_out`, `Section_External_ID`
 - `Centroid_name`: should be unique & consistent with the ["OD Matrix"](#od) data. You may name it as `I1`, `I2`, `E1`, `E2`, etc. 
 - `Centroid_Longitude`, `Centroid_Latitude`: should be EPSG values
 - `Generate_Section_External_ID`: a list of section externel IDs of generating direction
 - `Absort_Section_External_ID`:a list of section externel IDs of absorbing direction

In [4]:
fake_cendroid_file = pd.DataFrame({'Centroid_name':['I1','I2', 'E1'],
                                   'Centroid_Longitude':list(1000*np.random.rand(3)), 
                                   'Centroid_Latitude':list(1000*np.random.rand(3)), 
                                   'Generate_Section_External_ID':[[1000, 5215, 3353], [2534, 3463, 1234], [9529]],
                                  'Absort_Section_External_ID':[[1001, 5214, 3354], [2537, 3462, 1235], [9531]]})
fake_cendroid_file

,Centroid_name,Centroid_Longitude,Centroid_Latitude,Generate_Section_External_ID,Absort_Section_External_ID
0,I1,583.449821,107.839864,"[1000, 5215, 3353]","[1001, 5214, 3354]"
1,I2,697.515730,382.048570,"[2534, 3463, 1234]","[2537, 3462, 1235]"
2,E1,451.027340,893.949075,[9529],[9531]


In Aimsun, using scripts to generate centroids located in the corresponding longitude & latitude, and links the centroids to the corresponding sections according to External_IDs (two directions: generates to & attracts from)

```python
''' This is a Python 2 code, can be executed only in Aimsun '''

NAME = 0
LON = 2
LAT = 3
FROM = 5
TO = 4

import csv

def create_new_configuration():
    cmd = model.createNewCmd(model.getType("GKCentroidConfiguration"))
    model.getCommander().addCommand(cmd)
    return cmd.createdObject()

def create_new_centroid(configuration, name, Lon, Lat):
    cmd = model.createNewCmd(model.getType("GKCentroid"))
    cmd.setData(GKPoint(Lon, Lat), configuration)
    model.getCommander().addCommand(cmd)
    centroid = cmd.createdObject()
    centroid.setName(name)
    return centroid

def create_new_connection(source_object, destination_object):
    cmd = GKCenConnectionNewCmd()
    cmd.setData(source_object, destination_object)
    model.getCommander().addCommand(cmd)
    return cmd.createdObject()

def get_object(external_id):
    return model.getCatalog().findObjectByExternalId(external_id)

def main():
    config = Object = model.getCatalog().find(60507)
    #config = create_new_configuration()

    with open('C:\Users\Theo\Dropbox\Aimsun\General Data\Aimsun Project Dependency File\centroid_connections.csv') as linksfile:
        csvreader = csv.reader(linksfile, delimiter=",")
        next(csvreader)

        for row in csvreader:
            centroid_name = row[NAME]
            Lon = float(row[LON])
            Lat = float(row[LAT])
            print(centroid_name, Lon, Lat)
            centroid = create_new_centroid(config, centroid_name, Lon, Lat)

            if len(row[FROM]) != 0:
                from_sections = row[FROM].split(", ")
                print(from_sections)
                for from_section_id in from_sections:
                    from_section = get_object(from_section_id)
                    create_new_connection(from_section, centroid)
            
            if len(row[TO]) != 0:
                to_sections = row[TO].split(", ")
                print(to_sections)
                for to_section_id in to_sections:
                    to_section = get_object(to_section_id)
                    create_new_connection(centroid, to_section)
		
            centroid.setPositionByConnections()
    print "Done"

main()
```

<a id='od'></a>
### 2. Import the OD matrix as demand data.   <font color=red size=3>Data Format for Data Collection Team Reference</font>   
- Two csv files for **commuter** and several csv files for **resident**.
- The centroid_name should be unique & consistent with the ["Centroid data"](#centroid)

In [5]:
resident = pd.DataFrame({'CentroidID_O':['int_1','int_1', 'int_5'],
                         'CentroidID_D':['int_2','ext_1','int_1'], 
                         'Time':['2000-01-01 14:00:00+00:00','2000-01-01 14:15:00+00:00','2000-01-01 14:30:00+00:00'], 
                         'Count':[15,12,10]})
print("This is an example for resident, we also need a similar file for commuter")
resident

This is an example for resident, we also need a similar file for commuter


,CentroidID_O,CentroidID_D,Time,Count
0,int_1,int_2,2000-01-01 14:00:00+00:00,15
1,int_1,ext_1,2000-01-01 14:15:00+00:00,12
2,int_5,int_1,2000-01-01 14:30:00+00:00,10


Using scripts to import OD matrices into Aimsun

```python
''' This is a Python 2 code, can be executed only in Aimsun '''

import csv
import time
### PARAMETERS BEGIN
# Column location in the CSV file
FROM = 1
TO = 2
TIME = 3
TIME_START = 11
TIME_END = 16
COUNT = 4

SKIP_CSV_HEAD = 1
# Aimsun Vehicle type ID
RESIDENT = 63068
COMMUTER = 63069
vehId = RESIDENT

# Aimsun Centroid Configuration ID
CENTROID_CONFIG = 60507

TIME_DURATION = GKTimeDuration(00,15,00)
### PARAMETERS END

centroid_config_id = CENTROID_CONFIG
centroidConf = model.getCatalog().find(centroid_config_id)
        
def find_matrix_object(time_str, OD_dict):
    if time_str in OD_dict:
        return OD_dict[time_str]
    else:
        # load the matrix to centroid configurations
        matrix = GKSystem.getSystem().newObject("GKODMatrix", model)
        matrix_name_veh = 'Commuter' if vehId == COMMUTER else 'Resident'
        matrix_name = matrix_name_veh + ' - ' + time_str
        matrix.setName(matrix_name)
        centroidConf.addODMatrix(matrix)
        # set vehicle ID
        vehicle = centroidConf.getModel().getCatalog().find(vehId)
        matrix.setVehicle(vehicle)
        # set time
        matrix.setFrom(QTime.fromString(time_str, "hh:mm"))
        matrix.setDuration(TIME_DURATION)
    
        # add matrix to the dictionary
        OD_dict[time_str] = matrix
    return OD_dict[time_str]
    
def import_od_to_aimsun(od_file):
    """
    This function loads the OD matrix to AIMSUN from the OD matrix file
    :param od_file: the OD matrix file in AIMSUN required format
    :para od_time: int tuple (hour, minute)
    :param vehId: int vehicle ID
    :return: the demand
    """
    OD_dict = {}
    empty_centroid = set()
    for line in open(od_file, "r").readlines()[SKIP_CSV_HEAD:]:
        tokens = line.split(",")
        # find OD matrix
        time_str = tokens[TIME][TIME_START:TIME_END]
        matrix = find_matrix_object(time_str, OD_dict)
        
        # extract values from line
        fromCentroid = centroidConf.getModel().getCatalog().findByName(str(tokens[FROM]))
        toCentroid = centroidConf.getModel().getCatalog().findByName(str(tokens[TO]))
        trips = float(tokens[COUNT])
	    # Set the value if the section is valid
        if fromCentroid == None or toCentroid == None:
            if fromCentroid == None:
                empty_centroid.add(tokens[FROM])
            if toCentroid == None:
                empty_centroid.add(tokens[TO])
            continue
        matrix.setTrips(fromCentroid, toCentroid, trips)
    
    for name in empty_centroid:
        print("Warning: centroid " + name + " not found")
    print(' Done!')

    
od_file = '/Users/jinhengxu/Desktop/internal_OD_demand.csv'
import_od_to_aimsun(od_file)
```

<a id='detector'></a>
### 3. Import Detectors.  <font color=red size=3>Data Format for Data Collection Team Reference</font>   

- It is one csv file for all detectors and their locations.
- The detector_name should be unique & consistent with the ["Real Data Set"](#rds)
    - If each road section has no more than one detector, you may name detector as "detector_{section_ID}", e.g. "detector_6594"
- You may add more information for detector locations, but it is not necessary
    - position of the detector
    - length of the detector
    - covered lanes of the detector

In [80]:
detector_file = pd.DataFrame({'Detector Name':['Detector_6594','Detector_9423','Detector_5243'],
                              'Located Section externel ID':[6594,9423,5243],
                             })
detector_file

,Detector Name,Located Section externel ID
0,Detector_6594,6594
1,Detector_9423,9423
2,Detector_5243,5243


Using scripts to add detectors into Aimsun

```python
''' This is a Python 2 code, can be executed only in Aimsun '''

import csv
import os


def createDetector( model, section, name ):
    #Creates new GKDetector object.
    detector = GKSystem.getSystem().newObject( "GKDetector", model )
    #Sets parameters.
    detector.setName( name )
    detector.setLanes( 0, len(section.getLanes()) - 1 ); # we can change them, now we set the detector covers all the lanes in this section
    detector.setLength( 4.5 ); # we can change them
    detector.setPosition( section.length2D() / 2.0 );
    #Adds detector to model.
    section.addTopObject( detector );
    model.getGeoModel().add( section.getLayer(), detector );

def get_object(eid):
    return model.getCatalog().findObjectByExternalId(eid)

# Entry code, the script starts here:

file=open('C:\Users\Theo\Fremont Dropbox\Theophile Cabannes\Aimsun\Weekly project 20 Spring\Week 23\detectors\detectors_to_road_segments_2019.csv')
reader = csv.reader(file)

# skip the first line
line = next(reader)

for line in reader:

    name = 'detector' + str(line[0]) # line[0] should be detector id 
    print('detector name is %s' % name)

    target = get_object(str(int(line[1]))) #  external id of the road section

    if target != None:
#         target.setName('d'+line[0])
        print('section name is %s' % target.getName())
        createDetector(model, target, name)
        # Be sure that you reset the UNDO buffer after a modification that cannot be undone
        model.getCommander().addCommand( None )
        print("Finish creating detectors on %d" % int(line[1]))
    else:
        model.reportError("Create Detector", "The script must be launched from a Section context menu")

print('Finish adding detectors')
```

<a id='rds'></a>
### 4. Import Real Data Detectors.  <font color=red size=3>Data Format for Data Collection Team Reference</font>  
    
- It is one csv file for all detectors and their data collected in different time.
- The detector_name should unique & be consistent with the ["Detector"](#detector) data 
- Vehicle count is the most important data, we might need speed data 
in dynamic simulation later

In [88]:
real_data = pd.DataFrame({'Detector Name':['Detector_6594']*4+['Detector_9423']*4,
                              'Time data collected':['14:00:00', '14:15:00', '14:30:00', '14:45:00', '14:00:00', '14:15:00', '14:30:00', '14:45:00'],
                             'Vehicle Type':['car']*8, 
                             'Vehicle Count in this peroid': list(1000*np.random.rand(8)),
                             'Vehicle Speed in this peroid': list(100*np.random.rand(8))})
real_data

,Detector Name,Time data collected,Vehicle Type,Vehicle Count in this peroid,Vehicle Speed in this peroid
0,Detector_6594,14:00:00,car,748.465800,82.823591
1,Detector_6594,14:15:00,car,154.248259,59.791796
2,Detector_6594,14:30:00,car,13.680613,11.985300
3,Detector_6594,14:45:00,car,522.690176,38.363366
4,Detector_9423,14:00:00,car,427.663566,93.178511
5,Detector_9423,14:15:00,car,759.809021,10.855887
6,Detector_9423,14:30:00,car,187.217716,14.641316
7,Detector_9423,14:45:00,car,898.185803,62.565367


    a. In Aimsun: Project -> New -> Data Analysis -> Real Data Set    
    b. Double click the new real data set, add an Real Data Simple File Reader. Then open it, set the columns and import Detectors CSV Files.

<a id='control'></a>
### 5. Add Traffic Control Plan.    
    a. Find the location of the intersection and double click the location, then select “Signal Groups” in the popup window.    
    b. Click the “New” button and choose “Turns” corresponding to a certain phase of signal. Repeat this step and add all the turns.
    c. Right click the intersection and choose “Edit Control Plan” -> “Control Plan”. In the popup window, set “type” as “fixed”, set “Cycle” as the traffic signal cycle.    
    d. Click “Add Phase” and Assign it to a “Signal”. Drag the Green bar to set the green time of the Signal. Click “Add Phase” and select “Interphase” to add yellow + red time. 

***
# Part 3 Simulation

### 1. Edit View Modes for visualization.    
    a. In Aimsun: edit/add items in View Modes.     
    b. To show different colors for vehicles: double click “PARAMETERS: Vehicle Type” in View Modes, edit “Used Styles”. Select Objects of Type as “Simulation Vehicles”, Attribute: “Vehicle Type”. Style: “Color”. Then click “New” and set Symbol as desired color and set Value as the name of vehicle type.    
    c. We can follow similar steps to show different colors for other attributes, e.g. colors for maximum section speed, color for real-time road capacity, etc.

### 2. Do Static OD Adjustment Simulation    
    a. In Aimsun, Project -> New -> Scenarios -> Static OD Adjustment Scenario    
    b. Double click the new Scenario, choose Traffic Demand to be adjusted and Detection Data as the ground truth.    
    c. Create a new experiment and run. After adjustment, click “Create Demand and Matrices” to generate a new Traffic Demand holding the adjusted matrices.

### 3. Do Static Assignment Scenario    
    a. In Aimsun: Project -> New -> Demand Data -> Path Assignment, which is where we save the paths from the static assignment that we might want to use later as an input in other dynamic scenarios.    
    b. Project -> New -> Scenarios -> Static Assignment Scenario. Open the scenario and select the above Path Assignment in the “Output to Generates” tab.    
    c. Create a new experiment and run.

### 4. Do Dynamic OD Adjustment (To Do)

### 5. Set Routing Behavior, Car Following/Lane Changing Behavior    
    a. In Aimsun: Vehicles -> Choose any vehicle type and double click -> Vehicle Type Editor -> Microscopic Model    
    b. Change the parameters of the Car-following Model (sensitivity factor and gap) 

### 6. Do Micro Simulation    
    a. In Aimsun: Scenarios -> Dynamic Scenario (double click) -> Main     
    b. Select the traffic demand matrix and the master control plan    
    c. Scenarios -> Dynamic Scenario -> Micro SRC Experiment -> Replication (right click) -> Run Animated Simulation

### 7. Redo Simulation Using Replay
    a. In Aimsun: Scenarios -> Dynamic Senario -> Micro SRC Experiment -> Replication (right click) -> Play Recorded Simulation

***
# Part 4 Validation

### 1. Validation can be done using:   
    a. Pass Travel Time       
    b. Pass Flow    
    c. Link Flow
    d. Link Speed

### 2. SQLite File 
    a. Automatically genereted after simulation      
    b. .sqlite file can be found under the same folder with .ang file